In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import os
from random import randrange
from scipy.optimize import curve_fit
from scipy.optimize import OptimizeWarning
from scipy.stats import chi2
import warnings
import sys
import colorcet as cc
import multiprocessing

sys.path.append(os.path.join(os.path.dirname(
    os.path.abspath(''))))
import common.plots as plots
import common.potential_data as potential_data
import common.fit as fit

warnings.simplefilter("ignore", OptimizeWarning)
warnings.simplefilter("ignore", RuntimeWarning)

In [11]:
def potential_T_dependence_gluodynamics(lattice_size, L, beta, smearing, additional_params, decomposition_type, smearing_max, copy=None):
    # paths = [{'path': f'../../result/potential/smearing/wilson_loop/fundamental/on-axis/su3/gluodynamics/{lattice_size}/beta{beta}/{smearing}/{additional_params}/potential_{decomposition_type}.csv',
    #           'parameters': {'beta': f'beta={beta}'}, 'constraints': {'space_size': (1, L // 2), 'time_size': (1, L // 4 * 3), 'smearing_step': (0, smearing_max)}}]
    paths = [{'path': f'../../result/potential/smearing/wilson_loop/fundamental/on-axis/su3/gluodynamics/{lattice_size}/beta{beta}/{smearing}/{additional_params}/potential_{decomposition_type}.csv',
              'parameters': {'beta': f'beta={beta}'}, 'constraints': {'smearing_step': (0, smearing_max)}}]
    if copy is not None:
        paths[0]['constraints'] = {'copy': (copy, copy)}
    image_path = f'../../images/smearing/potential/su3/gluodynamics/T_dependence/{lattice_size}/beta{beta}/{smearing}/{additional_params}/{decomposition_type}'
    image_name = f'potential_abelian_smearing'
    data = potential_data.get_potantial_df(paths)
    print(data)
    for r in data['space_size'].unique():
        fg = plots.make_plot(data[data['space_size'] == r], 'time_size', 'aV(r)', 'smearing_step', 'T', 'aV(r)', f'{decomposition_type} T_dependence r={r}', image_path, f'potential_{decomposition_type}_T_dependence_r={r}', False, err='err', color_palette=cc.glasbey)
        plots.save_image(image_path, f'potential_{decomposition_type}_T_dependence_r={r}', fg)
        plt.close()

In [12]:
potential_T_dependence_gluodynamics('36^4', 36, '6.3', 'HYP0_APE_alpha=0.4', 'steps_0/copies=20', 'monopole', 151, 0)
potential_T_dependence_gluodynamics('36^4', 36, '6.3', 'HYP0_APE_alpha=0.45', 'steps_0/copies=20', 'monopole', 151, 0)
potential_T_dependence_gluodynamics('36^4', 36, '6.3', 'HYP0_APE_alpha=0.5', 'steps_0/copies=20', 'monopole', 151, 0)

       copy  smearing_step  space_size  time_size     aV(r)       err  \
0         0              0           1          1  0.003886  0.000005   
1         0              0           1          2  0.004311  0.000006   
2         0              0           1          3  0.004539  0.000006   
3         0              0           1          4  0.004684  0.000006   
4         0              0           1          5  0.004782  0.000006   
...     ...            ...         ...        ...       ...       ...   
27715     0            201          36         31 -0.549620  0.001707   
27716     0            201          36         32 -0.530158  0.001219   
27717     0            201          36         33 -0.507146  0.000937   
27718     0            201          36         34 -0.474069  0.000764   
27719     0            201          36         35 -0.430025  0.000642   

           beta  
0      beta=6.3  
1      beta=6.3  
2      beta=6.3  
3      beta=6.3  
4      beta=6.3  
...         ...

In [3]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP0_alpha=1_1_0.5_APE_alpha=0.3', 'steps_500/copies=4', 'monopole', 151)
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP0_alpha=1_1_0.5_APE_alpha=0.4', 'steps_500/copies=4', 'monopole', 151)

In [4]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP0_alpha=1_1_0.5_APE_alpha=0.5', 'steps_500/copies=4', 'monopole', 151)
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP0_alpha=1_1_0.5_APE_alpha=0.6', 'steps_500/copies=4', 'monopole', 151)

In [5]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP0_alpha=1_1_0.5_APE_alpha=0.7', 'steps_500/copies=4', 'monopole', 151)
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP0_alpha=1_1_0.5_APE_alpha=0.8', 'steps_500/copies=4', 'monopole', 151)

In [6]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP0_alpha=1_1_0.5_APE_alpha=0.9', 'steps_500/copies=4', 'monopole', 151)

In [7]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP0_alpha=1_1_0.5_APE_alpha=1', 'steps_500/copies=4', 'monopole', 151)

In [8]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP1_alpha=1_1_0.5_APE_alpha=0.3', 'steps_500/copies=4', 'monopole', 151)
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP1_alpha=1_1_0.5_APE_alpha=0.4', 'steps_500/copies=4', 'monopole', 151)

In [9]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP1_alpha=1_1_0.5_APE_alpha=0.5', 'steps_500/copies=4', 'monopole', 151)
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP1_alpha=1_1_0.5_APE_alpha=0.6', 'steps_500/copies=4', 'monopole', 151)

In [10]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP1_alpha=1_1_0.5_APE_alpha=0.7', 'steps_500/copies=4', 'monopole', 151)
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP1_alpha=1_1_0.5_APE_alpha=0.8', 'steps_500/copies=4', 'monopole', 151)

In [11]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP1_alpha=1_1_0.5_APE_alpha=0.9', 'steps_500/copies=4', 'monopole', 151)

In [12]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP1_alpha=1_1_0.5_APE_alpha=1', 'steps_500/copies=4', 'monopole', 151)

In [13]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP3_alpha=1_1_0.5_APE_alpha=0.3', 'steps_500/copies=4', 'monopole', 151)
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP3_alpha=1_1_0.5_APE_alpha=0.4', 'steps_500/copies=4', 'monopole', 151)

In [14]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP3_alpha=1_1_0.5_APE_alpha=0.5', 'steps_500/copies=4', 'monopole', 151)
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP3_alpha=1_1_0.5_APE_alpha=0.6', 'steps_500/copies=4', 'monopole', 151)

In [15]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP3_alpha=1_1_0.5_APE_alpha=0.7', 'steps_500/copies=4', 'monopole', 151)
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP3_alpha=1_1_0.5_APE_alpha=0.8', 'steps_500/copies=4', 'monopole', 151)

In [16]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP3_alpha=1_1_0.5_APE_alpha=0.9', 'steps_500/copies=4', 'monopole', 151)

In [17]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP3_alpha=1_1_0.5_APE_alpha=1', 'steps_500/copies=4', 'monopole', 151)

In [3]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP0_alpha=1_1_0.5_APE_alpha=0.3', 'steps_500/copies=4', 'monopoless', 151)
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP0_alpha=1_1_0.5_APE_alpha=0.4', 'steps_500/copies=4', 'monopoless', 151)

In [4]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP0_alpha=1_1_0.5_APE_alpha=0.5', 'steps_500/copies=4', 'monopoless', 151)
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP0_alpha=1_1_0.5_APE_alpha=0.6', 'steps_500/copies=4', 'monopoless', 151)

In [5]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP0_alpha=1_1_0.5_APE_alpha=0.7', 'steps_500/copies=4', 'monopoless', 151)
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP0_alpha=1_1_0.5_APE_alpha=0.8', 'steps_500/copies=4', 'monopoless', 151)

In [6]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP0_alpha=1_1_0.5_APE_alpha=0.9', 'steps_500/copies=4', 'monopoless', 151)

In [7]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP0_alpha=1_1_0.5_APE_alpha=1', 'steps_500/copies=4', 'monopoless', 151)

In [8]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP1_alpha=1_1_0.5_APE_alpha=0.3', 'steps_500/copies=4', 'monopoless', 151)
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP1_alpha=1_1_0.5_APE_alpha=0.4', 'steps_500/copies=4', 'monopoless', 151)

In [9]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP1_alpha=1_1_0.5_APE_alpha=0.5', 'steps_500/copies=4', 'monopoless', 151)
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP1_alpha=1_1_0.5_APE_alpha=0.6', 'steps_500/copies=4', 'monopoless', 151)

In [10]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP1_alpha=1_1_0.5_APE_alpha=0.7', 'steps_500/copies=4', 'monopoless', 151)
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP1_alpha=1_1_0.5_APE_alpha=0.8', 'steps_500/copies=4', 'monopoless', 151)

In [11]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP1_alpha=1_1_0.5_APE_alpha=0.9', 'steps_500/copies=4', 'monopoless', 151)

In [12]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP1_alpha=1_1_0.5_APE_alpha=1', 'steps_500/copies=4', 'monopoless', 151)

In [13]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP3_alpha=1_1_0.5_APE_alpha=0.3', 'steps_500/copies=4', 'monopoless', 151)
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP3_alpha=1_1_0.5_APE_alpha=0.4', 'steps_500/copies=4', 'monopoless', 151)

In [14]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP3_alpha=1_1_0.5_APE_alpha=0.5', 'steps_500/copies=4', 'monopoless', 151)
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP3_alpha=1_1_0.5_APE_alpha=0.6', 'steps_500/copies=4', 'monopoless', 151)

In [15]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP3_alpha=1_1_0.5_APE_alpha=0.7', 'steps_500/copies=4', 'monopoless', 151)
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP3_alpha=1_1_0.5_APE_alpha=0.8', 'steps_500/copies=4', 'monopoless', 151)

In [16]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP3_alpha=1_1_0.5_APE_alpha=0.9', 'steps_500/copies=4', 'monopoless', 151)

In [17]:
potential_T_dependence_gluodynamics('24^4', 24, '6.0', 'HYP3_alpha=1_1_0.5_APE_alpha=1', 'steps_500/copies=4', 'monopoless', 151)